# Formalities #



## Imports

In [25]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from timeit import default_timer as timer
from numba import jit, njit, vectorize

## Custom Plot Colormap ##

In [26]:
plt.style.use('seaborn-white')
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

## Trig Functions in Degrees



``` 
math.sin(angle*math.pi/180)
math.cos(angle*math.pi/180)
math.tan(angle*math.pi/180)
```



# Velocity and Angle Effect on Height of Ball at Target Distance

##Constants##


In [27]:
Ub = 5 # Upper Bound of Target
Lb = 4 # Lower Bound of Target
Ht = 4.5 # Midpoint of Target
Hs = 0.5 # Height of Shooter
Dt = 3 # Distance to Target
Db = 0.18 # Diameter of Ball
halfGrav = -9.81/2

In [28]:
angle=np.arange(1, 90, 0.1)
velocity=np.arange(0.01, 25.01, 0.01)
X, Y = np.meshgrid(angle, velocity)

## Function ##

In [29]:
@vectorize
def findHeightAtTarget(angle, velocity):
  t = Dt/(velocity*math.cos(angle*math.pi/180))
  return (halfGrav*(t**2))+(velocity*math.sin(angle*math.pi/180)*t)+(Hs)

In [30]:
@vectorize
def vecCondition(item):
  u = -Db/2
  b = Db/2
  if item<b and item>u:
      return 0
  return 1

In [31]:
findHeightAtTarget(45, 10)

2.6170999999999998

## Plotting

In [19]:
Z = vecCondition(findHeightAtTarget(X, Y)-Dt)
%matplotlib widget
plt.contourf(X, Y, Z, 50, norm=MidpointNormalize(midpoint=0.), cmap='seismic')
plt.colorbar();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testing

## Speed of Function ##

In [32]:
def std(aList, bList):
  result = np.zeros(shape=(len(aList),len(bList)))
  for a in range(len(aList)):
    for b in range(len(bList)):
      result[a][b]=findHeightAtTarget(aList[a], bList[b])
  return result

In [35]:
%time Z = findHeightAtTarget(X, Y)

Wall time: 35.9 ms


In [36]:
%time std(angle, velocity)

Wall time: 5.79 s


array([[-4.41583948e+05, -1.10395573e+05, -4.90643922e+04, ...,
         4.81598493e-01,  4.81655118e-01,  4.81711675e-01],
       [-4.41612195e+05, -1.10402631e+05, -4.90675260e+04, ...,
         4.86831714e-01,  4.86888342e-01,  4.86944902e-01],
       [-4.41643135e+05, -1.10410362e+05, -4.90709592e+04, ...,
         4.92064853e-01,  4.92121486e-01,  4.92178050e-01],
       ...,
       [-1.61023117e+10, -4.02557750e+09, -1.78914524e+09, ...,
        -2.00704455e+03, -2.00497974e+03, -2.00291741e+03],
       [-3.62300179e+10, -9.05750382e+09, -4.02555678e+09, ...,
        -4.94615582e+03, -4.94151002e+03, -4.93686979e+03],
       [-1.44919632e+11, -3.62299066e+10, -1.61021798e+10, ...,
        -2.15049137e+04, -2.14863306e+04, -2.14677697e+04]])

In [37]:
del X, Y, angle, velocity

## Plotting

In [7]:
a=np.arange(25, 90.01, 0.1, dtype="float32")
b=np.arange(7.5, 25.1, 0.01, dtype="float32")
X, Y = np.meshgrid(a, b)
Z = vecCondition(findHeightAtTarget(X, Y)-Dt)
#Z = findHeightAtTarget(X, Y)-Dt
%matplotlib widget 
plt.contourf(X, Y, Z, 50, norm=MidpointNormalize(midpoint=0.), cmap='seismic')
plt.show()
plt.colorbar()
#print(X)
#print(Y)
#print(Z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Speed of Condiontal Formatting ##

In [49]:
def normalCondition(array):
    u = -0.5
    b = 0.5
    l = len(array[0])
    for i in range(len(array)):
        for j in range(l):
            if array[i][j]<b and array[i][j]>u:
                array[i][j]=0
            else:
                array[i][j]=1

In [50]:
%time vecCondition(Z)

Wall time: 7.98 ms


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int64)

In [51]:
%time normalCondition(Z)

Wall time: 2.16 s
